In [ ]:
!pip3 install torch==1.13.0 torchvision torchaudio torchtext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 79.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 113.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 121.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 15.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. Thi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile, os, urllib.request, glob, math, shutil, sys, random
import torchvision
import torch.optim as optim
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, precision_recall_fscore_support, roc_auc_score, accuracy_score
import sklearn.metrics
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import PIL
import PIL.Image

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive


Extract IND and OOD images.

In [ ]:
os.makedirs('train')
os.makedirs('test')

prefix = 'drive/MyDrive/ood_classification/'

for i in ['train', 'test']:
  for j in ['ind', 'ood']:

    name = i + '_' + j
    src_zip = prefix + name + '.zip'
    dest_zip = i + '/' + j

    with zipfile.ZipFile(src_zip, 'r') as zip_ref:
      zip_ref.extractall(dest_zip)

    print("Finished with: ", name)

!ls train/ind | wc -l
!ls test/ind | wc -l
!ls train/ood | wc -l
!ls test/ood | wc -l

Finished with:  train_ind
Finished with:  train_ood
Finished with:  test_ind
Finished with:  test_ood
1839
459
1839
459


Κάνω load το μοντέλο που έγινε train χωρίς τα μετα-δεδομένα. Στη συνέχεια ορίζω το τελικό νευρωνικό δίκτυο(metaModel) που θα γίνει train στις εικόνες + μετα-δεδομένα.

In [ ]:
"""
from torchvision.models import mobilenet_v3_large

model = mobilenet_v3_large(weights = 'DEFAULT')
model.classifier[3] = nn.Linear(1280, out_features = 256, bias = True)
#prostheto ena akoma linear layer gia to teleytaio classifier
model.classifier.add_module("4", nn.Linear(256, 2, bias = True))
print("Test output vector: ", model(torch.rand(2,3,224,224)))
model.to(device)

"""
from torchvision.models import mobilenet_v3_small
model = mobilenet_v3_small(weights = 'DEFAULT')
model.classifier[3] = nn.Linear(1024, out_features = 2, bias = True)
print("Test output vector: ", model(torch.rand(2,3,224,224)))
model.to(device)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


  0%|          | 0.00/9.83M [00:00<?, ?B/s]

Test output vector:  tensor([[-0.0609, -0.0299],
        [-0.1314,  0.3091]], grad_fn=<AddmmBackward0>)


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

Ορίζω διάφορα transformations για να κάνουμε augment το training set, ώστε να πετύχουμε καλύτερο generalization του μοντέλου. Εφαρμόζω πιο ισχυρά transformations από αυτά που εφάρμοσα στα training των ISIC,PAD.

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomResizedCrop(size = 224, scale = (0.4,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness = [0.5, 1.5], contrast = [0.5, 1.5], saturation = [0.5, 1.5], hue = [-0.2,0.2]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transforms_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

Ορισμός υπερ-παραμέτρων.

In [ ]:
epochs = 15
batch_len = 64

criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma=0.2)

Εδώ ορίζω τα dataset και dataloaders, χρησιμοποιώντας το weightRandomSampler για να δημιουργήσει balanced batches:

In [ ]:
train_dataset = datasets.ImageFolder('train', transforms_train)
test_dataset = datasets.ImageFolder('test', transforms_test)

len_train_dataset = len(train_dataset)
len_test_dataset = len(test_dataset)

print("Length of train_dataset: ", len_train_dataset)
print("Length of test dataset: ", len_test_dataset)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_len, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_len, shuffle=False)

Length of train_dataset:  3678
Length of test dataset:  918


Ορίζω συναρτήσεις για υπολογισμό του accuracy και του confusion matrix πάνω στο test set. Στη συνέχεια θα δημιουργήσω και άλλες συναρτήσεις για υπολογισμό F1-score, sensitivity, specificity, precision.

In [ ]:
def test_model(model,test_dataloader):

  test_acc = 0.0
  pred_list = []
  label_list = []
  output_probs = []
  metrics = []

  for i, (inputs, labels) in enumerate(test_dataloader,1):

    label_list.extend(labels.data.tolist())

    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    probs = torch.softmax(outputs.data.cpu(), dim = 1)[:,1].tolist()
    output_probs.extend(probs)

    pred =  torch.max(outputs, 1)[1].data.cpu().tolist()
    pred_list.extend(pred)

  conf_matrix = confusion_matrix(label_list , pred_list)

  metrics.append(accuracy_score(label_list, pred_list))
  metrics.extend(precision_recall_fscore_support(label_list,pred_list, average = 'macro'))
  metrics.append(roc_auc_score(label_list,output_probs, average = 'macro', multi_class = 'ovr'))

  return metrics, conf_matrix

Το βασικό train loop.

In [ ]:
for epoch in range(1, epochs + 1):

  model.train()
  train_running_loss = 0

  #Training phase
  print("Starting epoch: " , epoch)

  for i, (inputs, labels) in enumerate(train_dataloader,1):
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      train_running_loss += loss.detach().item()

  print('Epoch: %d | Loss: %.4f ' %(epoch, train_running_loss / i))
  scheduler.step()

  model.eval()
  try:
    metrics,conf_matrix = test_model(model,test_dataloader)
    print('Test metrics on epoch %d:' %(epoch))
    print(metrics)
    print("Confusion matrix: ")
    print(conf_matrix)
  except Exception as e:
    print("Error in calculating metrics")
    print(e)

  try:
    checkpoint = {'model': model.state_dict(), 'epoch' : epoch}
    checkpoint_fn = 'drive/MyDrive/ood_classification/checkpoint' + str(epoch) + '.pt'
    torch.save(checkpoint, checkpoint_fn)
  except Exception as e:
    print("Error in writing general checkpoint")
    print(e)

Δημιουργία για app.

In [ ]:
print(torch.__version__)
print(torchvision.__version__)

ood_classifier_app = mobilenet_v3_small()
ood_classifier_app.classifier[3] = nn.Linear(1024, out_features = 2, bias = True)

ood_classifier_app.load_state_dict(torch.load('drive/MyDrive/ood_classification/checkpoint9.pt', map_location=torch.device("cpu"))['model'])
ood_classifier_app.eval()

traced_script_module = torch.jit.trace(ood_classifier_app, torch.rand(1,3,224,224))
traced_script_module._save_for_lite_interpreter("drive/MyDrive/ood_classification/ood_classifier.ptl")

1.13.0+cu117
0.14.0+cu117


Τεστ τον DataLoader.

In [ ]:
from collections import Counter

for inputs, labels in train_dataloader:
  inputs = inputs.to(device)
  print("Input size:", inputs.shape)
  out = model(inputs)
  print("Output size: ", out.shape)

  labels = labels.tolist()
  print("Test if batch is balanced: ")
  print(Counter(labels).keys())
  print(Counter(labels).values())
  break

Input size: torch.Size([64, 3, 224, 224])
Output size:  torch.Size([64, 2])
Test if batch is balanced: 
dict_keys([0, 1])
dict_values([34, 30])
